In [ ]:
%conf 
numRows=10 
truncate=100

In [ ]:
%env 
ETL_CONF_JOB_URL=/home/jovyan/examples/movie_graph
ETL_CONF_BASE_URL=https://raw.githubusercontent.com/arangodb/example-datasets/master/Graphs

In [ ]:
{
  "type": "HTTPExtract",
  "name": "get nodes",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_CONF_BASE_URL}"/IMDB/nodes.json",
  "outputView": "nodes_raw",
  "persist": true
}

In [ ]:
{
  "type": "SQLTransform",
  "name": "split the body",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_CONF_JOB_URL}"/split_table.sql",
  "outputView": "nodes_split",
  "sqlParams": {
    "inputView": "nodes_raw"
  }
}

In [ ]:
{
  "type": "JSONExtract",
  "name": "users",
  "environments": ["production", "test"],
  "inputView": "nodes_split",
  "inputField": "col",
  "outputView": "nodes",
  "persist": true
}

In [ ]:
%sql outputView=person
SELECT 
  _key AS id
  ,label AS name
  ,_key AS key
FROM nodes
WHERE type = 'Person'

In [ ]:
%sql outputView=movie
SELECT 
  _key AS id
  ,label AS name
  ,_key AS key
  ,from_unixtime(releaseDate/1000) AS releaseDate
FROM nodes
WHERE type = 'Movie'

In [ ]:
{
  "type": "HTTPExtract",
  "name": "get edges",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_CONF_BASE_URL}"/IMDB/edges.json",
  "outputView": "edges_raw",
  "persist": true  
}

In [ ]:
{
  "type": "SQLTransform",
  "name": "split the body",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_CONF_JOB_URL}"/split_table.sql",
  "outputView": "edges_split",
  "sqlParams": {
    "inputView": "edges_raw"
  }
}

In [ ]:
{
  "type": "JSONExtract",
  "name": "users",
  "environments": ["production", "test"],
  "inputView": "edges_split",
  "inputField": "col",
  "outputView": "edges",
  "persist": true
}

In [ ]:
%sql outputView=directed
SELECT DISTINCT
  _key AS id
  ,_from as source
  ,_to AS target
FROM edges
WHERE `$label`='DIRECTED'

In [ ]:
%sql outputView=acts_in
SELECT DISTINCT
  _key AS id
  ,_from as source
  ,_to AS target
FROM edges
WHERE `$label`='ACTS_IN'

In [ ]:
{
  "type": "GraphTransform",
  "name": "create imdb graph",
  "environments": [
    "production",
    "test"
  ],
  "nodes": [
    {"label": "Person", "view": "person"},
    {"label": "Movie", "view": "movie"},
  ],
  "relationships": [
    {"type": "ACTS_IN", "view": "acts_in"},
    {"type": "DIRECTED", "view": "directed"}
  ],
  "outputGraph": "imdb",
  "persist": true
} 

In [ ]:
%cypher
FROM session.imdb
MATCH (p0:Person)-[:ACTS_IN]->(m:Movie)<-[:ACTS_IN]-(p1:Person)
WHERE p0.name = 'Robin Williams'
AND p0.key <> p1.key
RETURN p0.name, p1.name
ORDER BY m.releaseDate